In [1]:
from neo4j import GraphDatabase
uri="bolt://neo4j:7687"; user="neo4j"; pwd="ChangeMe!123"
drv = GraphDatabase.driver(uri, auth=(user, pwd))
with drv.session() as s:
    print(s.run("MATCH (n) RETURN count(n) AS nodes").single())

<Record nodes=211>


In [2]:
with drv.session() as s:
    q = "MATCH (p:Person)-[:KNOWS]->(f) RETURN p.name,f.name LIMIT 10"
    for rec in s.run(q):
        print(rec[0], "->", rec[1])

Alice -> Bob
Bob -> Carol


In [3]:
with drv.session() as s:
    q = """
    MATCH (p:Person)
    RETURN p.name AS person, size([(p)-[:KNOWS]->() | 1]) AS out_degree
    ORDER BY out_degree DESC
    """
    print(s.run(q).data())

[{'person': 'Alice', 'out_degree': 1}, {'person': 'Bob', 'out_degree': 1}, {'person': 'Carol', 'out_degree': 0}]


In [4]:
with drv.session() as s:
    s.run("CALL gds.graph.drop('sg', false) YIELD graphName").consume()
    s.run("""
      CALL gds.graph.project('sg',
        'Person', {KNOWS: {orientation: 'NATURAL'}}
      )
    """).consume()
    res = s.run("""
      CALL gds.degree.stream('sg')
      YIELD nodeId, score
      RETURN gds.util.asNode(nodeId).name AS person, score
      ORDER BY score DESC
    """).data()

res[:5]

[{'person': 'Alice', 'score': 1.0},
 {'person': 'Bob', 'score': 1.0},
 {'person': 'Carol', 'score': 0.0}]